In [4]:
!which pip

/home/wschung1113/anaconda3/envs/jeju_specialty/bin/pip


In [5]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import sys
import holidays


import sklearn
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


import xgboost
from xgboost import XGBRegressor
import catboost
from catboost import CatBoostRegressor

from optuna.trial import Trial
import optuna

/home/wschung1113/anaconda3/envs/jeju_specialty/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(f"파이썬 버전 : {sys.version}")
print(f"pandas 버전 : {pd.__version__}")
print(f"numpy 버전 : {np.__version__}")
print(f"sklearn 버전 : {sklearn.__version__}")
print(f"xgboost 버전 확인 : {xgboost.__version__}")
print(f"catboost 버전 : {catboost.__version__}")
print(f"optuna 버전 : {optuna.__version__}")

파이썬 버전 : 3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
pandas 버전 : 2.2.1
numpy 버전 : 1.26.4
sklearn 버전 : 1.3.0
xgboost 버전 확인 : 1.7.3
catboost 버전 : 1.2
optuna 버전 : 3.5.0


In [73]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2024) # Seed 고정

In [74]:
train = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/train.csv')
test = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/test.csv')
international_data = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/international_trade.csv')

In [75]:
international_data

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


In [76]:
print(international_data['품목명'].value_counts())

품목명
감귤                                                                               50
포포(papaw)[파파야(papaya)]                                                           50
레몬[시트러스 리몬(Citrus limon)ㆍ시트러스 리머늄(Citrus limonum)]                               50
그레이프프루트(grapefruit)와 포멜로(pomelo)                                                 50
오렌지                                                                              50
망고(mango)                                                                        50
파인애플                                                                             50
무화과                                                                              50
대추야자                                                                             50
사과                                                                               50
오이류(신선한 것이나 냉장한 것으로 한정한다)                                                        50
당근                                                                      

In [77]:
print(international_data['품목명'].unique())

['토마토(신선한 것이나 냉장한 것으로 한정한다)' '양파' '쪽파' '꽃양배추와 브로콜리(broccoli)' '방울다다기 양배추'
 '양배추' '결구(結球) 상추'
 '위트루프 치커리(Witloof chicory)[시코리엄 인티부스 변종 포리오섬(Cichorium intybus var. foliosum)]'
 '당근' '오이류(신선한 것이나 냉장한 것으로 한정한다)' '콩[비그나(Vigna)속ㆍ파세러스(Phaseolus)속)]'
 '대추야자' '무화과' '파인애플' '망고(mango)' '망고스틴(mangosteen)' '오렌지' '감귤'
 '그레이프프루트(grapefruit)와 포멜로(pomelo)'
 '레몬[시트러스 리몬(Citrus limon)ㆍ시트러스 리머늄(Citrus limonum)]' '수박'
 '포포(papaw)[파파야(papaya)]' '사과' '배' '자두' '완두[피섬 새티범(Pisum sativum)]'
 '구아바(guava)' '살구' '신 체리[프루너스 체라서스(Prunus cerasus)]'
 '복숭아[넥터린(nectarine)을 포함한다]' '순무' '방울토마토' '샬롯(shallot)' '참외']


In [78]:
relevant_dict = {"TG": "감귤", "BC": "꽃양배추와 브로콜리(broccoli)", "RD": None, "CR": "당근", "CB": "양배추"} # RD: 무는 없음

In [79]:
international_data.columns

Index(['기간', '품목명', '수출 중량', '수출 금액', '수입 중량', '수입 금액', '무역수지'], dtype='object')

In [80]:
new_column_names = {"기간": "timestamp", "품목명": "item", "수출 중량": "export_weight", "수출 금액": "export_amount", "수입 중량": "import_weight", "수입 금액": "import_amount", "무역수지": "trade_profit"}
international_data.rename(columns=new_column_names, inplace=True)

In [81]:
international_data['timestamp'] = pd.to_datetime(international_data['timestamp'])

In [82]:
type(international_data.timestamp[0])

pandas._libs.tslibs.timestamps.Timestamp

In [83]:
def change_type(type_):
    result = None
    for key, value in relevant_dict.items():
            if value == type_:
                result = key
                break
    return result

In [84]:
international_data['item'] = international_data.apply(lambda x: change_type(x['item']), axis=1)

In [85]:
# data leakage를 피하기 위해 international_data는 2023년 1월분까지만 사용
print(international_data.shape)
international_data = international_data.loc[international_data.timestamp < pd.to_datetime("2023-02-01"), ]
print(international_data.shape)

(1274, 7)
(1246, 7)


In [86]:
scaler = StandardScaler()
international_data.loc[:, ["export_weight", "export_amount", "import_weight", "import_amount", "trade_profit"]] = scaler.fit_transform(international_data.loc[:, ["export_weight", "export_amount", "import_weight", "import_amount", "trade_profit"]])

/tmp/ipykernel_136475/2630015994.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.15734056  0.63386664 -0.20819682 ...  1.27812049 -0.20825321
 -0.20825834]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  international_data.loc[:, ["export_weight", "export_amount", "import_weight", "import_amount", "trade_profit"]] = scaler.fit_transform(international_data.loc[:, ["export_weight", "export_amount", "import_weight", "import_amount", "trade_profit"]])
/tmp/ipykernel_136475/2630015994.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.36833914 -0.08320562 -0.21314233 ...  2.27270565 -0.21373028
 -0.21373028]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  international_data.loc[:, ["export_weight", "export_amount", "import_weight", "import_amount", "trad

In [87]:
international_data

,timestamp,item,export_weight,export_amount,import_weight,import_amount,trade_profit
0,2019-01-01,None,0.157341,0.368339,-0.340814,-0.263856,0.350580
1,2019-01-01,None,0.633867,-0.083206,0.739002,-0.074244,0.047591
2,2019-01-01,None,-0.208197,-0.213142,-0.315619,-0.242147,0.171132
3,2019-01-01,BC,-0.208094,-0.213142,-0.168475,-0.168371,0.101249
4,2019-01-01,None,-0.208258,-0.213730,-0.338769,-0.257411,0.185430
...,...,...,...,...,...,...,...
1241,2023-01-01,None,-0.208258,-0.213730,-0.337957,-0.258089,0.186073
1242,2023-01-01,None,-0.094759,-0.043225,-0.340814,-0.263856,0.238124
1243,2023-01-01,None,1.278120,2.272706,-0.340814,-0.263856,0.870932
1244,2023-01-01,None,-0.208253,-0.213730,-0.340814,-0.263856,0.191535


In [88]:
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    # 합쳐서 전처리하기
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)
    
    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    #년/월/일 추가
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #요일 추가
    df['week_day']=df['timestamp'].dt.weekday

    # 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
    le = LabelEncoder()
    df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

    # 라벨 인코딩
    df["year_month"] = le.fit_transform(df["year_month"])


    # 주차 변수 추가
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    # 주차 누적값
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    # 공휴일 변수 추가
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0
        
    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))

    # 국제 무역 데이터 접합하기
    # subsidiary_data = []
    # for i in range(len(df)):
    #     row = df.iloc[i, :]
    #     item_ = row['item']
    #     row_date = row.timestamp
    #     target = relevant_dict[item_]
    #     if target == None:
    #         subsidiary_data.append([None] * 5)
    #         continue
    #     target_row = international_data.loc[(international_data.type == relevant_dict["TG"]) & (international_data.date_year_month == row_date), ["export_weight", "export_amount", "import_weight", "import_amount", "trade_profit"]]
    #     if target_row.empty:
    #         subsidiary_data.append([None] * 5)
    #         continue
    #     target_row_list = target_row.values.tolist()[0]
    #     subsidiary_data.append(target_row_list)

    # subsidiary_df = pd.DataFrame(subsidiary_data)

    # print(df.shape)
    # print(subsidiary_df.shape)

    # df = pd.concat([df, subsidiary_df], axis=1)
    # df = pd.merge(df, international_data, on=["timestamp", "item"], how="inner")
    df = pd.merge(df, international_data, on=["timestamp", "item"], how="left")

    # train, test 분리하기
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

In [89]:
train_pre, test_pre = pre_all(train, test)

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 20)
전처리 후 test 크기 : (1092, 20)


In [90]:
print(train_pre)
print(train_pre.columns)

                    ID  timestamp item corporation location    supply   price  \
0      TG_A_J_20190101 2019-01-01   TG           A        J       0.0     0.0   
1      CB_A_S_20190101 2019-01-01   CB           A        S       0.0     0.0   
2      RD_D_J_20190101 2019-01-01   RD           D        J       0.0     0.0   
3      BC_D_J_20190101 2019-01-01   BC           D        J       0.0     0.0   
4      CB_F_J_20190101 2019-01-01   CB           F        J       0.0     0.0   
...                ...        ...  ...         ...      ...       ...     ...   
59392  CR_E_S_20230303 2023-03-03   CR           E        S       0.0     0.0   
59393  BC_A_S_20230303 2023-03-03   BC           A        S    3776.0  2875.0   
59394  CB_E_J_20230303 2023-03-03   CB           E        J       0.0     0.0   
59395  BC_D_J_20230303 2023-03-03   BC           D        J    1776.0  3059.0   
59396  RD_F_J_20230303 2023-03-03   RD           F        J  427520.0   529.0   

       year  month  day  we

In [91]:
print(test_pre)

                   ID  timestamp item corporation location  supply  price  \
0     TG_A_J_20230304 2023-03-04   TG           A        J     NaN    NaN   
1     TG_E_S_20230304 2023-03-04   TG           E        S     NaN    NaN   
2     BC_B_J_20230304 2023-03-04   BC           B        J     NaN    NaN   
3     TG_E_J_20230304 2023-03-04   TG           E        J     NaN    NaN   
4     BC_B_S_20230304 2023-03-04   BC           B        S     NaN    NaN   
...               ...        ...  ...         ...      ...     ...    ...   
1087  TG_A_J_20230331 2023-03-31   TG           A        J     NaN    NaN   
1088  RD_D_J_20230331 2023-03-31   RD           D        J     NaN    NaN   
1089  CR_D_J_20230331 2023-03-31   CR           D        J     NaN    NaN   
1090  TG_E_J_20230331 2023-03-31   TG           E        J     NaN    NaN   
1091  RD_F_J_20230331 2023-03-31   RD           F        J     NaN    NaN   

      year  month  day  week_day  year_month  week  week_num  holiday  \
0 

# 0. TG 외 품목들
### 0-1. 전처리

In [92]:
## 전처리

# 극 이상치 제거
tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()


# 감귤이 아닌것
print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_notg = train_pre[train_pre["item"] !="TG"]
test_notg = test_pre[test_pre["item"] != "TG"]


#인코딩
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])
print(Xy.columns)

train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday', 'export_weight', 'export_amount', 'import_weight',
       'import_amount', 'trade_profit'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday', 'export_weight', 'export_amount', 'import_weight',
       'import_amount', 'trade_profit'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'export_weight',
       'export_amount', 'import_weight', 'import_amount', 'trade_profit',
       'item_BC', 'item_CB', 'item_CR', 'item_RD', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'corporation_F', 'location_J', 'loc

In [93]:
def objective_vote(trial: Trial, model_1, model_2, x, y, eval_metric):
    """
    Optuna의 하이퍼파라미터 튜닝을 위한 목적 함수
    
    Parameters:
    - trial: Optuna의 Trial 객체
    - model: 튜닝할 머신러닝 모델
    - x: 입력 데이터
    - y: 타겟 데이터
    - eval_metric: 모델을 평가하기 위한 메트릭
    """
    
    # 모델에 맞는 하이퍼파라미터 범위를 정의합니다.
    # cat = CatBoostRegressor(random_state = 2024, 
    #                             n_estimators = n_estimators, 
    #                             learning_rate = lrs, 
    #                             depth = max_depths, 
    #                             l2_leaf_reg = l2_leaf_reg,
    #                             metric_period = 1000)
    # xgb = XGBRegressor(n_estimators = 1000,
    #                    random_state = 2024,
    #                    learning_rate = 0.01,
    #                    max_depth = 10)
    params_cat = {
        'random_state': 2024,
        'metric_period': 1000,
        'thread_count': -1,
        # 'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error']),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, step=0.01),
        'depth': trial.suggest_int('depth', 5, 15, step=1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 5, step=1),
        # 'max_features': trial.suggest_float('max_features', 0.6, 0.9, step=0.1),
    }

    params_xgb = {
        'random_state': 2024,
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, step=0.01),
        'max_depth': trial.suggest_int('depth', 5, 15, step=1),
    }

    params_voting = {
        'weights': [trial.suggest_float('learning_rate', 0, 1, step=0.1), trial.suggest_float('learning_rate', 0, 1, step=0.1)]
    }

    # 데이터 프레임이나 시리즈 형태의 x와 y를 numpy 배열로 변환합니다.
    x_np = x.values if isinstance(x, pd.DataFrame) else x    
    y_np = y.values if isinstance(y, pd.Series) else y

    results = dict()
    
    # 5-Fold Cross Validation을 정의합니다.
    # fold = KFold(n_splits=5, shuffle=True, random_state=2024)
    
    # 각 Fold에 대해 모델을 학습하고 평가합니다.
    # for i, (train_idx, test_idx) in enumerate(fold.split(x, y)):
    years = [2019, 2020, 2021, 2022]
    for i, year in enumerate(years):
        filtered_df = x[(x['year'] == year) & (x['month'] == 3) & (x['day'].between(4, 31))]
        train_idx = x.index.difference(filtered_df.index)
        test_idx = filtered_df.index

        x_train, y_train = x_np[train_idx], y_np[train_idx]
        x_test, y_test = x_np[test_idx], y_np[test_idx]
        
        # 현재의 하이퍼파라미터 세트로 모델을 생성하고 학습합니다.
        # voting
        cat = model_1(**params_cat)
        xgb = model_2(**params_xgb)
        fold_model = VotingRegressor(
            estimators =[("cat",cat), ("xgb", xgb)],
            voting='soft',
            **params_voting,
        )

        fold_model.fit(x_train, y_train)
        fold_pred = fold_model.predict(x_test)
        
        # 음수 추론값 0으로 치환
        for idx in range(len(fold_pred)):
            if fold_pred[idx]<0:
                fold_pred[idx]= 0

        
        # 해당 Fold의 평가 결과를 저장합니다.
        fold_error = eval_metric(y_test, fold_pred)
        
        results[i] = {
            'model': fold_model, 
            'error': fold_error
        }
    
    # 모든 Fold의 평가 결과의 평균을 반환합니다.
    errors = [v['error'] for k, v in results.items()]
    return np.array(errors).mean()

In [94]:
# 최소화 방향으로 하이퍼 파라미터 학습을 위한 스터디 객체를 생성합니다.
study_voting = optuna.create_study(direction='minimize', study_name='voting_regressor_1')

# 하이퍼파라미터 튜닝을 시작합니다.
study_voting.optimize(lambda trial: objective_vote(trial,
                                                   CatBoostRegressor,
                                                   XGBRegressor,
                                                   Xy.drop(columns = ["timestamp", "ID","price"]),
                                                   Xy["price"],
                                                   mean_squared_error),
                                                   n_trials=20)

In [95]:
# 최적의 하이퍼파라미터를 출력합니다.
print(study_voting.best_params)

# # 모델에 적용시
# model = RandomForestRegressor(**study.best_params)
# model.fit(x, y)

In [96]:
## 앙상블 모델 정의

# cat = CatBoostRegressor(random_state = 2024, 
#                             n_estimators = 1000, 
#                             learning_rate = 0.01, 
#                             depth = 10,
#                             l2_leaf_reg = 3,
#                             metric_period = 1000)
# cat = CatBoostRegressor(random_state = 2024, 
#                             metric_period = 1000,
#                             **study.best_params)

# xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)
cat = CatBoostRegressor()
xgb = XGBRegressor()

# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID", "price"]), Xy["price"])

pred = vote_model.predict(answer_notg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_notg["answer"] = pred

answer_notg[["ID","answer"]]

0:	learn: 923.0041829	total: 7.31ms	remaining: 5.84s
799:	learn: 368.9378603	total: 4.61s	remaining: 0us


,ID,answer
2,BC_B_J_20230304,2508.147215
4,BC_B_S_20230304,183.390632
6,BC_C_J_20230304,2514.466105
7,BC_A_S_20230304,2798.988935
10,BC_D_J_20230304,2909.215552
...,...,...
1085,RD_D_S_20230331,484.292780
1086,CR_C_J_20230331,1773.018243
1088,RD_D_J_20230331,444.345799
1089,CR_D_J_20230331,1939.259611


# 1. TG (1)
### 1-1. 전처리

In [97]:
train_pre, test_pre = pre_all(train, test)

# 공휴일이지만 안쉬는 날 제외하기
no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0

# train 및 test 시간 순서로 정렬하기
train_tg = train_pre[train_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)
test_tg = test_pre[test_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_tg1.columns}")
Xy["price"] = np.sqrt(Xy["price"])

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 20)
전처리 후 test 크기 : (1092, 20)
train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'export_weight',
       'export_amount', 'import_weight', 'import_amount', 'trade_profit',
       'item_TG', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday', 'export_weight', 'export_amount',
       'import_weight', 'import_amount', 'trade_profit', 'item_TG',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'location_J', 'location_S'],
      dtype='object')


### 3-2. 모델링 & 훈련 예측

In [98]:
# # 모델 정의
# cat = CatBoostRegressor(random_state = 2024, 
#                             n_estimators = 1000, 
#                             learning_rate = 0.01, 
#                             depth = 10,
#                             l2_leaf_reg = 3,
#                             metric_period = 1000)
cat = CatBoostRegressor(random_state = 2024, 
                            metric_period = 1000,
                            **study.best_params)
xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_tg1.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg1["answer"] = np.power(pred,2)

answer_tg1[["ID","answer"]]

0:	learn: 30.6073679	total: 5.04ms	remaining: 4.02s
799:	learn: 14.9303818	total: 3.22s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2735.490800
1,TG_E_S_20230304,3380.409401
2,TG_E_J_20230304,593.513615
3,TG_D_S_20230304,3636.158665
4,TG_D_J_20230304,177.123287
...,...,...
275,TG_D_J_20230331,1665.146813
276,TG_D_S_20230331,5173.276573
277,TG_A_S_20230331,5553.466512
278,TG_E_S_20230331,4919.463870


# 4. TG (2)
- 일반화를 위한 추가 모델링
### 4-1. 전처리

In [99]:
train_tg2 = train_pre[train_pre["item"] =="TG"]
test_tg2 = test_pre[test_pre["item"] == "TG"]

Xy2 = pd.get_dummies(train_tg2.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg2 = pd.get_dummies(test_tg2.drop(columns = ["timestamp","supply","price","item"]), columns = [ "corporation","location"])
print(Xy2.columns)

# 종속변수 루트값
Xy2["price"] = np.sqrt(Xy2["price"])

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'export_weight',
       'export_amount', 'import_weight', 'import_amount', 'trade_profit',
       'corporation_A', 'corporation_B', 'corporation_C', 'corporation_D',
       'corporation_E', 'location_J', 'location_S'],
      dtype='object')


### 4-2. 모델링 & 훈련 예측

In [ ]:
def objective_cat(trial: Trial, model, x, y, eval_metric):
    """
    Optuna의 하이퍼파라미터 튜닝을 위한 목적 함수
    
    Parameters:
    - trial: Optuna의 Trial 객체
    - model: 튜닝할 머신러닝 모델
    - x: 입력 데이터
    - y: 타겟 데이터
    - eval_metric: 모델을 평가하기 위한 메트릭
    """
    
    # 모델에 맞는 하이퍼파라미터 범위를 정의합니다.
    # cat = CatBoostRegressor(random_state = 2024, 
    #                             n_estimators = n_estimators, 
    #                             learning_rate = lrs, 
    #                             depth = max_depths, 
    #                             l2_leaf_reg = l2_leaf_reg,
    #                             metric_period = 1000)
    params = {
        'random_state': 2024,
        'thread_count': -1,
        # 'criterion': trial.suggest_categorical('criterion', ['squared_error', 'absolute_error']),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, step=0.01),
        'depth': trial.suggest_int('depth', 5, 15, step=1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 5, step=1),
        # 'max_features': trial.suggest_float('max_features', 0.6, 0.9, step=0.1),
    }

    # 데이터 프레임이나 시리즈 형태의 x와 y를 numpy 배열로 변환합니다.
    x_np = x.values if isinstance(x, pd.DataFrame) else x    
    y_np = y.values if isinstance(y, pd.Series) else y

    results = dict()
    
    # 5-Fold Cross Validation을 정의합니다.
    # fold = KFold(n_splits=5, shuffle=True, random_state=2024)
    
    # 각 Fold에 대해 모델을 학습하고 평가합니다.
    # for i, (train_idx, test_idx) in enumerate(fold.split(x, y)):
    years = [2019, 2020, 2021, 2022]
    for i, year in enumerate(years):
        filtered_df = x[(x['year'] == year) & (x['month'] == 3) & (x['day'].between(4, 31))]
        train_idx = x.index.difference(filtered_df.index)
        test_idx = filtered_df.index

        x_train, y_train = x_np[train_idx], y_np[train_idx]
        x_test, y_test = x_np[test_idx], y_np[test_idx]
        
        # 현재의 하이퍼파라미터 세트로 모델을 생성하고 학습합니다.
        fold_model = model(**params)  
        fold_model.fit(x_train, y_train)
        fold_pred = fold_model.predict(x_test)
        
        # 해당 Fold의 평가 결과를 저장합니다.
        fold_error = eval_metric(y_test, fold_pred)
        
        results[i] = {
            'model': fold_model, 
            'error': fold_error
        }
    
    # 모든 Fold의 평가 결과의 평균을 반환합니다.
    errors = [v['error'] for k, v in results.items()]
    return np.array(errors).mean()

In [100]:
# 모델 정의 및 훈련 예측
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

# cat = CatBoostRegressor(random_state = 2024, 
#                                 n_estimators = n_estimators, 
#                                 learning_rate = lrs, 
#                                 depth = max_depths, 
#                                 l2_leaf_reg = l2_leaf_reg,
#                                 metric_period = 1000)
cat = CatBoostRegressor(random_state = 2024, 
                            metric_period = 1000,
                            **study.best_params)
cat.fit(Xy2.drop(columns = ["timestamp", "ID","price"]), Xy2["price"])

pred2 = cat.predict(answer_tg2.drop(columns = ["ID"]))
for idx in range(len(pred2)):
    if pred2[idx]<0:
        pred2[idx]= 0
answer_tg2["answer"] = np.power(pred2,2)

answer_tg2[["ID","answer"]]

0:	learn: 30.6122496	total: 7.17ms	remaining: 5.73s
799:	learn: 14.9019513	total: 3.53s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2817.542087
1,TG_E_S_20230304,3409.830275
3,TG_E_J_20230304,1019.689881
5,TG_D_S_20230304,3646.742081
8,TG_D_J_20230304,576.703029
...,...,...
1074,TG_D_S_20230331,5132.000731
1077,TG_A_S_20230331,5425.052000
1079,TG_E_S_20230331,4941.007349
1087,TG_A_J_20230331,5020.509523


# 5. TG 앙상블

In [101]:
total1 = pd.concat([answer_tg1[["ID","answer"]],answer_notg[["ID","answer"]]])
total2 = pd.concat([answer_tg2[["ID","answer"]],answer_notg[["ID","answer"]]])

In [102]:
print(total1.shape)
print(total2.shape)

(1092, 2)
(1092, 2)


In [103]:
# TG 앙상블 (평균)

df = pd.merge(total1, total2, how = "inner", on="ID")

In [104]:
df

,ID,answer_x,answer_y
0,TG_A_J_20230304,2735.490800,2817.542087
1,TG_E_S_20230304,3380.409401,3409.830275
2,TG_E_J_20230304,593.513615,1019.689881
3,TG_D_S_20230304,3636.158665,3646.742081
4,TG_D_J_20230304,177.123287,576.703029
...,...,...,...
1087,RD_D_S_20230331,484.292780,484.292780
1088,CR_C_J_20230331,1773.018243,1773.018243
1089,RD_D_J_20230331,444.345799,444.345799
1090,CR_D_J_20230331,1939.259611,1939.259611


In [105]:

df["answer"] = (df["answer_x"]+df["answer_y"])/2
df["item"] = df["ID"].map(lambda x :x.split("_")[0])

In [106]:
# 후처리 
## 전체 min값 | 3월의 min값 확인
df.loc[(df['item']=='TG')&(df['answer']<400),'answer'] =0 # 551   #3월 675
df.loc[(df['item']=='CB')&(df['answer']<50),'answer'] =0 # 162  # 3월 200
df.loc[(df['item']=='RD')&(df['answer']<10),'answer'] =0 # 50     # 3월 124
df.loc[(df['item']=='CR')&(df['answer']<150),'answer'] =0 # 250   # 3월 450
df.loc[(df['item']=='BC')&(df['answer']<100),'answer'] =0 #205 3월 205.0

In [107]:
df = df.drop(columns = ["answer_x","answer_y", "item"])
df

,ID,answer
0,TG_A_J_20230304,2776.516443
1,TG_E_S_20230304,3395.119838
2,TG_E_J_20230304,806.601748
3,TG_D_S_20230304,3641.450373
4,TG_D_J_20230304,0.000000
...,...,...
1087,RD_D_S_20230331,484.292780
1088,CR_C_J_20230331,1773.018243
1089,RD_D_J_20230331,444.345799
1090,CR_D_J_20230331,1939.259611


In [108]:
# 제출용
df.to_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/answer.csv',index=False)